<a href="https://colab.research.google.com/github/hoops92/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/Scott_LS_DS11_232.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [0]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [0]:
!ls -alh

total 197M
drwxr-xr-x 1 root root 4.0K Jan 28 17:26 .
drwxr-xr-x 1 root root 4.0K Jan 28 17:24 ..
drwxr-xr-x 1 root root 4.0K Jan 13 16:38 .config
-rw-r--r-- 1 root root 197M Jan 28 17:26 instacart_online_grocery_shopping_2017_05_01.tar.gz
drwxr-xr-x 1 root root 4.0K Jan 13 16:38 sample_data


In [0]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [0]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

order_products__train.csv
orders.csv
order_products__prior.csv
aisles.csv
products.csv
departments.csv


In [0]:
!ls -alh

total 681M
drwxr-xr-x 2  502 staff 4.0K May  2  2017 .
drwxr-xr-x 1 root root  4.0K Jan 28 17:27 ..
-rw-r--r-- 1  502 staff  226 May  2  2017 ._aisles.csv
-rw-r--r-- 1  502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1  502 staff  226 May  2  2017 ._departments.csv
-rw-r--r-- 1  502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1  502 staff  226 May  2  2017 ._order_products__prior.csv
-rw-r--r-- 1  502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1  502 staff  226 May  2  2017 ._order_products__train.csv
-rw-r--r-- 1  502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1  502 staff  226 May  2  2017 ._orders.csv
-rw-r--r-- 1  502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1  502 staff  226 May  2  2017 ._products.csv
-rw-r--r-- 1  502 staff 2.1M May  2  2017 products.csv


### For each csv file, look at its shape & head 

In [0]:
import pandas as pd
from IPython.display import display

def preview():
  for filename in glob('*.csv'):
    df = pd.read_csv(filename)
    print('\n', filename, df.shape)
    display(df.head())

preview()


 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0



 order_products__prior.csv (32434489, 4)


...

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [0]:
# What tables do I need to read in?
prior = pd.read_csv('order_products__prior.csv')

In [0]:
prior['product_id'].describe()

count    3.243449e+07
mean     2.557634e+04
std      1.409669e+04
min      1.000000e+00
25%      1.353000e+04
50%      2.525600e+04
75%      3.793500e+04
max      4.968800e+04
Name: product_id, dtype: float64

In [0]:
prior['product_id'].head()

0    33120
1    28985
2     9327
3    45918
4    30035
Name: product_id, dtype: int64

In [0]:
prior['product_id'].mode()  # Here it is - most commonly bought product!

0    24852
dtype: int64

In [0]:
prior['product_id'].value_counts()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
          ...  
11356         1
18001         1
6320          1
26268         1
30087         1
Name: product_id, Length: 49677, dtype: int64

In [0]:
# So this item is bought quite a lot
# How do we figure out what it is?

products = pd.read_csv('products.csv')
# products[products['product_id'] == 24852]

most_frequent_product_id = prior['product_id'].mode()[0]
products[products['product_id'] == most_frequent_product_id]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [0]:
# Let's check order_products__train.csv as well
train = pd.read_csv('order_products__train.csv')
train['product_id'].mode()

0    24852
dtype: int64

In [0]:
train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [0]:
train['order_id'].nunique()

131209

In [0]:
# Let's do our first merge - prior and products
# So we can more easily look up names of any other products being ordered

help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
    Merge DataFrame or named Series objects with a database-style join.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    
    Parameters
    ----------
    left : DataFrame
    right : DataFrame or named Series
        Object to merge with.
    how : {'left', 'right', 'outer', 'inner'}, default 'inner'
        Type of merge to be performed.
    
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order.
        * right: use only keys from right frame, similar to a SQL right outer join;
          preserve key 

In [0]:
prior = pd.merge(prior, products, on='product_id')

In [0]:
prior.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [0]:
prior.shape

(32434489, 7)

## How often are BANANAS included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [0]:
# We're using the train set, and want to see how often bananas are reordered
# We'll start by creating a 'bananas' column in the train set
train['bananas'] = train['product_id'] == 24852

In [0]:
train.head()

,order_id,product_id,add_to_cart_order,reordered,bananas
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False


In [0]:
# For each order, were any of the products bananas?
train.groupby('order_id')['bananas'].any()  # equivalent to x1 or x2 or... xn

order_id
1          False
36         False
38         False
96         False
98         False
           ...  
3421049    False
3421056    False
3421058    False
3421063    False
3421070    False
Name: bananas, Length: 131209, dtype: bool

In [0]:
# Save this as a dataframe
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()

In [0]:
train_wrangled.shape

(131209, 2)

In [0]:
# How many orders included bananas?
train_wrangled['bananas'].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [0]:
# Not directly!
# orders.csv has user_id and order_id
# order_products__prior.csv (and train) has order_id and product_id

# To start: in the order_products__prior table, which orders included bananas?
BANANAS = 24852
prior[prior.product_id==BANANAS]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1771449,10,24852,1,1,Banana,24,4
1771450,20,24852,6,0,Banana,24,4
1771451,22,24852,3,1,Banana,24,4
1771452,26,24852,2,1,Banana,24,4
1771453,52,24852,2,1,Banana,24,4
...,...,...,...,...,...,...,...
2244009,3421027,24852,3,1,Banana,24,4
2244010,3421030,24852,9,1,Banana,24,4
2244011,3421038,24852,2,0,Banana,24,4
2244012,3421078,24852,2,1,Banana,24,4


In [0]:
banana_prior_order_ids = prior[prior.product_id==BANANAS].order_id
banana_prior_order_ids.head()

1771449    10
1771450    20
1771451    22
1771452    26
1771453    52
Name: order_id, dtype: int64

In [0]:
# Look at the orders table, which orders included bananas?
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1625684,2717803,97609,prior,52,4,19,6.0
81029,3275604,4950,prior,14,2,9,7.0
2958840,2552983,178576,prior,11,1,11,10.0
941950,1821775,56500,prior,1,2,8,NaN
1844497,1337666,110722,prior,52,3,18,7.0


In [0]:
# In the orders table, which orders included bananas?
orders[orders.order_id.isin(banana_prior_order_ids)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14,738281,2,prior,4,2,10,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0
...,...,...,...,...,...,...,...
3420915,1764570,206202,prior,20,4,0,11.0
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0


In [0]:
# To explore/start, let's check a particular banana order - 738281
# And confirm that in the prior set it really does include bananas
prior[prior.order_id == 738281]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
1313559,738281,47209,9,0,Organic Hass Avocado,24,4
1873670,738281,24852,8,0,Banana,24,4
6583903,738281,37646,6,0,Organic Gala Apples,24,4
11211287,738281,36735,5,0,Organic Roasted Sliced Chicken Breast,96,20
14468499,738281,45613,11,0,Pad Thai,38,1
15253763,738281,34688,4,0,Good Belly Probiotics Juice Drink Blueberry Ac...,31,7
18855616,738281,32139,3,0,Hommus Classic Original,67,20
22264522,738281,21150,13,0,Fire Grilled Steak Bowl,38,1
23977423,738281,22829,7,0,Multigrain Flax Seeded Flatbread Crackers,78,19
24989071,738281,32792,2,1,Chipotle Beef & Pork Realstick,23,19


In [0]:
# Looks good, let's save the whole dataframe
banana_orders = orders[orders.order_id.isin(banana_prior_order_ids)]

In [0]:
banana_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14,738281,2,prior,4,2,10,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0


In [0]:
banana_orders.shape

(472565, 7)

In [0]:
# In the orders table, which users have bought bananas?
banana_user_ids = banana_orders.user_id.unique()
banana_user_ids[:5]

array([ 2, 10, 16, 21, 27])

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [0]:
# orders table, before removing non-banana users
orders.shape

(3421083, 7)

In [0]:
# Let's subset to just include orders for banana users
orders = orders[orders['userid'].isin(banana_user_ids)]
orders.shape

(1512975, 7)

In [0]:
# IDs of *all* the orders from customers who have *ever* bought bananas
subset_order_ids = orders.order_id.unique()

In [0]:
# order_products__prior table, shape before getting subset
prior.shape

(32434489, 7)

In [0]:
# order_products__prior table, shape after getting subset
prior = prior[prior.order_id.isin(subset_order_ids)]
prior.shape

(16534534, 7)

In [0]:
# order_products__train table, shape before getting subset
train.shape

(1384617, 5)

In [0]:
# order_products__train table, shape after getting subset
train = train[train.order_id.isin(subset_order_ids)]
train.shape

(587269, 5)

In [0]:
train.head()

,order_id,product_id,add_to_cart_order,reordered,bananas
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False


In [0]:
# In this subset, how often were bananas reordered in the customer's most recent order?
target = 'bananas'
train[target].value_counts(normalize=True)

False    0.971807
True     0.028193
Name: bananas, dtype: float64

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?

In [0]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
13,1901567,2,prior,3,1,10,3.0
14,738281,2,prior,4,2,10,8.0
15,1673511,2,prior,5,3,11,8.0


In [0]:
# Merge user_id, order_number, order_dow, order_hour_of_day, and days_since_prior_order
# with the training data
train = pd.merge(train, orders)

In [0]:
train.head()

,order_id,product_id,add_to_cart_order,reordered,bananas,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,False,112108,train,4,4,10,9.0
1,1,11109,2,1,False,112108,train,4,4,10,9.0
2,1,10246,3,0,False,112108,train,4,4,10,9.0
3,1,49683,4,0,False,112108,train,4,4,10,9.0
4,1,43633,5,1,False,112108,train,4,4,10,9.0


In [0]:
# We now finally have our dataframe that could be split into X/Y and trained
# To simplify/baseline, let's pick an individual user to explore and experiment

USER = 61911

In [0]:
prior = pd.merge(prior, orders[['order_id', 'user_id']])

In [0]:
prior['bananas'] = prior.product_id == BANANAS

In [0]:
user_prior = prior[prior.user_id == USER]
user_prior.shape

(196, 9)

In [0]:
user_prior['order_id'].nunique()

8

In [0]:
user_prior['product_id'].nunique()

89

In [0]:
# How many times have they ordered bananas?
user_prior['bananas'].sum()

6

In [0]:
user_prior[user_prior['bananas']]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,bananas
2759878,1579677,24852,6,0,Banana,24,4,61911,True
3634685,738971,24852,4,1,Banana,24,4,61911,True
3640671,837210,24852,6,1,Banana,24,4,61911,True
8657479,364645,24852,7,1,Banana,24,4,61911,True
9051757,768788,24852,6,1,Banana,24,4,61911,True
10511829,2271842,24852,1,1,Banana,24,4,61911,True


In [0]:
# What percentage of orders?
user_prior['bananas'].sum() / user_prior['order_id'].nunique()

0.75

*Challenge*

Continue to clean and explore your data. Can you engineer features to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you make a fast, first model that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. Here's a video walkthrough you can refer to if you get stuck or want hints!

https://youtu.be/pPWFw8UtBVg?t=584